<a href="https://colab.research.google.com/github/fecsaba/ComfyUI/blob/main/Flux1_dev_lora_init_load_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` and `INSTALL_GGUF_NODE` after first run if `USE_GOOGLE_DRIVE` is checked.
## Crystools

## EasyUse
A ComfyUI használhatóságának javítása érdekében számos gyakran használt csomóponthoz optimalizáltak és integráltak.
## ITools
Az iTools néhány életminőség-csomópont, mint például a kép létrehozásához használt lehetséges prompt beolvasása, a prompt fájlba mentése új sorként, a promptok beolvasása többsoros fájlból.
## Was node suite
Csomópontcsomag a ComfyUI-hoz számos új csomóponttal, például képfeldolgozással, szövegfeldolgozással és egyebekkel.

## ComfyUI's ControlNet Auxiliary Preprocessors
Plug-and-play ComfyUI csomópontkészletek ControlNet tippképek készítéséhez
"anime stílus, tiltakozás az utcán, cyberpunk város, egy rózsaszín hajú, arany szemű nő (nézi a nézőt) egy táblát tart a "ComfyUI ControlNet Aux" felirattal félkövér, neonrózsaszínnel a Flux.1 Dev-n
A kód másolás-beillesztése a https://github.com/lllyasviel/ControlNet/tree/main/annotator megfelelő mappáiból, és csatlakoztatva van a 🤗 Hubhoz .
## ComfyUI ControlAltAI Nodes
Fluxus mintavevő
A Flux Sampler csomópont egyetlen, áramvonalas csomópontban egyesíti a CustomSamplerAdvance csomópont és a bemeneti csomópontok funkcióit.

CFG beállítás: A CFG fix értéke 1.
Kondicionálás bemenet: Csak pozitív kondicionálás támogatott.
Kompatibilitás: Csak a Flux modellel kompatibilis mintavevőket és ütemezőket tartalmazza.
Látens kompatibilitás: Csak SD3 üres látens képet használjon. A normál üres látens képcsomópont nem kompatibilis.

## Rgthree Comfy !!!!

# INIT

In [4]:
# #@title Environment Setup
from pathlib import Path
import os

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_GGUF_NODE = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}

OPTIONS.update({
    'USE_GOOGLE_DRIVE': USE_GOOGLE_DRIVE,
    'UPDATE_COMFY_UI': UPDATE_COMFY_UI,
    'USE_COMFYUI_MANAGER': USE_COMFYUI_MANAGER,
    'INSTALL_GGUF_NODE': INSTALL_GGUF_NODE,
    'INSTALL_CUSTOM_NODES_DEPENDENCIES': INSTALL_CUSTOM_NODES_DEPENDENCIES
})

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

!export COMFYUI_PATH=WORKSPACE

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !chmod 755 .ci/**/*.bat .ci/**/*.py .ci/**/*.txt
  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes
  !chmod 755 ComfyUI-Manager/**/*.sh
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

if OPTIONS['INSTALL_GGUF_NODE']:
  %cd $WORKSPACE/custom_nodes
  !git clone https://github.com/city96/ComfyUI-GGUF
  !pip install -r ComfyUI-GGUF/requirements.txt

%cd $WORKSPACE

!echo -= Install custom nodes dependencies =-
!pip install GitPython
!python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

!echo -= Install nodes =-
!echo -= **************** =-
!echo -= Install Crystools =-
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/crystian/ComfyUI-Crystools.git
!pip install -r ComfyUI-Crystools/requirements.txt

!echo -= Install EasyUse =-
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/yolain/ComfyUI-Easy-Use.git
!pip install -r ComfyUI-Easy-Use/requirements.txt

!echo -= Install Itools =-

def install_custom_node(comfyui_path, repo_url, node_folder_name):
    custom_nodes_path = os.path.join(comfyui_path, "custom_nodes")

    if not os.path.exists(custom_nodes_path):
        os.makedirs(custom_nodes_path)
    print(f"custom_nodes mappa elérési útvonala: {custom_nodes_path}")

    destination_path = os.path.join(custom_nodes_path, node_folder_name)

    if not os.path.exists(destination_path):
        !git clone {repo_url} {destination_path}
        print(f"{node_folder_name} telepítése kész.")
    else:
        print(f"{node_folder_name} már telepítve van.")

    requirements_path = os.path.join(destination_path, "requirements.txt")
    if os.path.exists(requirements_path):
        !pip install -r {requirements_path}
        print("Dependenciák telepítve.")
    else:
        print("Nincs requirements.txt fájl, nem szükséges dependenciákat telepíteni.")

comfyui_path = "/content/drive/MyDrive/ComfyUI"
# iTools
repo_url = "https://github.com/fecsaba/ComfyUI-iTools.git"
node_folder_name = "ComfyUI-iTools"

install_custom_node(comfyui_path, repo_url, node_folder_name)

# Was node suite
repo_url = "https://github.com/WASasquatch/was-node-suite-comfyui.git"
node_folder_name = "was-node-suite-comfyui"

install_custom_node(comfyui_path, repo_url, node_folder_name)

# ComfyUI's ControlNet Auxiliary Preprocessors
repo_url = "https://github.com/Fannovel16/comfyui_controlnet_aux.git"
node_folder_name = "comfyui_controlnet_aux"

install_custom_node(comfyui_path, repo_url, node_folder_name)

# ComfyUI ControlAltAI Nodes
repo_url = "https://github.com/gseth/ControlAltAI-Nodes.git"
node_folder_name = "ControlAltAI-Nodes"

install_custom_node(comfyui_path, repo_url, node_folder_name)

%cd $WORKSPACE
print("Telepítés befejezve. Indítsd újra a ComfyUI-t!")

Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://download.pytorch.org/whl/cu121
/content/drive/MyDrive/ComfyUI/custom_nodes
/content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
Already up to date.
/content/drive/MyDrive/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI-GGUF' already exists and is not an empty directory.
/content/drive/MyDrive/ComfyUI
-= Install custom nodes dependencies =-

WARN: The `COMFYUI_PATH` environment variable is not set. Assuming 
`custom_nodes/ComfyUI-Manager/../../` as the ComfyUI path.
----------------------------------------------------------------------------------------------------
Restoring [1/8]: /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
Install: pip packages

## Comfy

### Download Resources

In [6]:
%cd $WORKSPACE
!mkdir -p "/content/models/"
# !wget -c "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/diffusion_models/hunyuan_video_t2v_720p_bf16.safetensors" -P "/content/models"
# !ln -s "/content/models/hunyuan_video_t2v_720p_bf16.safetensors" "/content/models/diffusion_models/hunyuan_video_t2v_720p_bf16.safetensors"
# Download the FLUX model from Hugging Face and save it in the ./models/unet directory
!wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q8_0.gguf?download=true" -P "/models"
!ln -s "/content/models/flux1-dev-Q8_0.gguf" "./content/models/unet/flux1-dev-Q8_0.gguf"

# Download the CLIP T5 model from Hugging Face and save it in ./models/clip
!wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q8_0.gguf?download=true" -P "/models"
!ln -s "/content/models/t5-v1_1-xxl-encoder-Q8_0.gguf" "./content/models/clip/t5-v1_1-xxl-encoder-Q8_0.gguf"

# Download the safetensors file for the CLIP model
!wget -c "https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l-for-gguf.safetensors?download=true" -P "/models"
!ln -s "/content/models/clip_l-for-gguf.safetensors" "./content/models/clip/clip_l-for-gguf.safetensors"

# Download the VAE model from Hugging Face and save it in ./models/vae
!wget -c "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors" -P "/models"
!ln -s "/content/models/ae.safetensors" "./content/models/vae/ae.safetensors"

# Download the LoRa model from Civitai.com Replace `298eebac3e9fecdcd7def0c8a41fa14d` with your actual token
# !wget -c "https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=298eebac3e9fecdcd7def0c8a41fa14d" -O "./models/loras/FluxThouS40k.safetensors"
# !wget -c "https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=298eebac3e9fecdcd7def0c8a41fa14d" -O "./models/loras/FluxThouS40k.safetensors"

# !mkdir -p "/content/models/"
# !wget -c "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/diffusion_models/hunyuan_video_t2v_720p_bf16.safetensors" -P "/content/models"
# !ln -s "/content/models/hunyuan_video_t2v_720p_bf16.safetensors" "/content/models/diffusion_models/hunyuan_video_t2v_720p_bf16.safetensors"

# !wget -c "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/clip_l.safetensors" -P "/content/models"
# !ln -s "/content/models/clip_l.safetensors" "/content/models/text_encoders/clip_l.safetensors"

# !wget -c "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/llava_llama3_fp8_scaled.safetensors" -P "/content/models"
# !ln -s "/content/models/llava_llama3_fp8_scaled.safetensors" "/content/models/text_encoders/llava_llama3_fp8_scaled.safetensors"

# !wget -c "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/vae/hunyuan_video_vae_bf16.safetensors" -P "/content/models"
# !ln -s "/content/models/hunyuan_video_vae_bf16.safetensors" "/content/models/vae/hunyuan_video_vae_bf16.safetensors"


/content/drive/MyDrive/ComfyUI
--2025-01-21 21:25:39--  https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q8_0.gguf?download=true
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.80, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/3e/5c/3e5c1f82226fc1460150126b5513fa266f4b8aea7194d1a31c944ad9da31ae10/129032f32224bf7138f16e18673d8008ba5f84c1ec74063bf4511a8bb4cf553d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27flux1-dev-Q8_0.gguf%3B+filename%3D%22flux1-dev-Q8_0.gguf%22%3B&Expires=1737498339&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNzQ5ODMzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzNlLzVjLzNlNWMxZjgyMjI2ZmMxNDYwMTUwMTI2YjU1MTNmYTI2NmY0YjhhZWE3MTk0ZDFhMzFjOTQ0YWQ5ZGEzMWFlMTAvMTI5MDMyZjMyMjI0YmY3MTM4ZjE2ZTE4NjczZDgwMDhiY

### Run ComfyUI

In [ ]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-01-21 21:32:51--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.1.0/cloudflared-linux-amd64.deb [following]
--2025-01-21 21:32:51--  https://github.com/cloudflare/cloudflared/releases/download/2025.1.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/2d8414e3-a676-4d22-b0ad-71fffb4149a4?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250121%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250121T213251Z&X-Amz-Expires=300&X-Amz-Signature=45e1dbfd3b8cb3fb3ed9700c5d6533fdd6db96408543b4e4b73f673376c8